In [1]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score

import matplotlib.pyplot as plt
%matplotlib inline

## 1. Предобработка данных

In [2]:
data = pd.read_csv('credit_scoring_sample.csv', sep=";")
data.head()

,SeriousDlqin2yrs,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,NumberOfTimes90DaysLate,NumberOfTime60-89DaysPastDueNotWorse,MonthlyIncome,NumberOfDependents
0,0,64,0,0.249908,0,0,8158.0,0.0
1,0,58,0,3870.000000,0,0,NaN,0.0
2,0,41,0,0.456127,0,0,6666.0,0.0
3,0,43,0,0.000190,0,0,10500.0,2.0
4,1,49,0,0.271820,0,0,400.0,0.0


In [3]:
def impute_nan_with_median(table):
    for col in table.columns:
        table[col]= table[col].fillna(table[col].median())
    return table 

In [4]:
data = impute_nan_with_median(data)

X = data.drop('SeriousDlqin2yrs', axis=1).values
y = data['SeriousDlqin2yrs'].values

**Задание 1.** Методом кросс-валидации по 5 блокам определите наилучшие значения `max_depth` и `max_features` для дерева решений. Используйте функции `GridSearchCV`, `StratifiedKFold` и `DecisionTreeClassifier` из sklearn. Укажите `class_weight='balanced'` для дерева решений. За метрику качества возьмите ROC AUC (`scoring='roc_auc'`). Какие значения гиперпараметров оказались лучшими? Какого качества ROC AUC удалось достичь?

In [5]:
max_depth_values = [5, 6, 7, 8, 9]
max_features_values = [4, 5, 6, 7]
tree_params = {'max_depth': max_depth_values,
               'max_features': max_features_values}

In [6]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [7]:
clf = DecisionTreeClassifier(random_state=42, class_weight="balanced")

In [8]:
gs_clf = GridSearchCV(clf,
                      param_grid=tree_params,
                      scoring='roc_auc',
                      n_jobs=4,
                      cv=kf)

In [9]:
gs_clf.fit(X, y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=DecisionTreeClassifier(class_weight='balanced',
                                              random_state=42),
             n_jobs=4,
             param_grid={'max_depth': [5, 6, 7, 8, 9],
                         'max_features': [4, 5, 6, 7]},
             scoring='roc_auc')

In [10]:
print(f"Best param: {gs_clf.best_params_}")
print(f"ROC AUC: {gs_clf.score(X, y)}")

Best param: {'max_depth': 6, 'max_features': 6}
ROC AUC: 0.8271760397877753


## 2. Реализация случайного леса

**Задание 2.** Используйте каркас модели, представленный ниже, для реализации случайного леса.

Краткая спецификация: 
- В методе `fit` в цикле (`i` от 0 до `n_estimators-1`) фиксируйте seed, равный (`random_state + i`).
- Зафиксировав seed, выберите **без замещения** `max_features` признаков, сохраните список выбранных id признаков в `self.feat_ids_by_tree`.
- Также сделайте bootstrap-выборку (т.е. **с замещением**) из множества id объектов.
- Обучите дерево с теми `max_depth`, `max_features` и `random_state`, которые указаны для класса, на выборке с нужным подмножеством объектов и признаков.
- В методе `predict_proba` опять нужен цикл по всем деревьям. У тестовой выборки нужно взять те признаки, на которых соответсвующее дерево обучалось, и сделать прогноз вероятностей (`predict_proba` уже для дерева). Метод должен вернуть усреднение прогнозов по всем деревьям (два столбца: вероятность 0 и вероятность 1).
- Метод `decision_function` должен возвращать вероятность того, что объект относится к классу 1.

Проведите кросс-валидацию с гиперпараметрами из предыдущего задания. Какое получилось среднее значение ROC AUC на кросс-валидации?

In [11]:
from sklearn.base import BaseEstimator

class RandomForestClassifierCustom(BaseEstimator):
    def __init__(self, n_estimators=10, max_depth=3, max_features=10, random_state=17):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.max_features = max_features
        self.random_state = random_state
        # в данный список необходимо записывать обученное дерево на каждой итерации
        self.trees = []
        # в данный список необходимо записывать индексы признаков, на которых обучалось дерево 
        self.feature_ids = []
        
    def fit(self, X, y):
        X_index_str = np.array(range(len(X)))
        X_index_col = np.array(range(len(X.transpose())))
        for i in range(self.n_estimators - 1):
            np.random.seed(self.random_state + i)
            # без замещения
            features =  np.random.choice(X_index_col, size=self.max_features, replace=False)
            self.feature_ids.append(features)
            # с замещением
            bootstrap_samples =  np.random.choice(X_index_str, size=len(X), replace=True)
            X_bootstrap = X[bootstrap_samples,:][:,features]
            y_bootstrap = y[bootstrap_samples]
            dt = DecisionTreeClassifier(max_depth=self.max_depth, max_features=self.max_features, random_state=self.random_state)
            dt.fit(X_bootstrap, y_bootstrap)
            self.trees.append(dt)
        
    def predict_proba(self, X):
        predict_proba_mean_value = 0
        for i, tree in enumerate(self.trees):
            features = self.feature_ids[i]
            X_test = X[:, features]
            predict_proba_mean_value += tree.predict_proba(X_test)
        predict_proba_mean_value /= self.n_estimators
        return predict_proba_mean_value
        
    def decision_function(self, X):
        return self.predict_proba(X)[:, 1]

In [12]:
rf = RandomForestClassifierCustom(max_depth=7, max_features=7)
rf.fit(X,y)

In [13]:
rf.predict_proba(X)

array([[0.86531345, 0.03468655],
       [0.83802018, 0.06197982],
       [0.78537862, 0.11462138],
       ...,
       [0.76209514, 0.13790486],
       [0.58701544, 0.31298456],
       [0.38951004, 0.51048996]])

In [14]:
rf.decision_function(X)

array([0.03468655, 0.06197982, 0.11462138, ..., 0.13790486, 0.31298456,
       0.51048996])

__Кросс-валидация__

In [15]:
rf_clf = RandomForestClassifierCustom(random_state=42)

In [16]:
gs_rf_clf = GridSearchCV(rf_clf,
                         param_grid=tree_params,
                         scoring='roc_auc',
                         n_jobs=4,
                         cv=kf)

In [17]:
gs_rf_clf.fit(X, y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=RandomForestClassifierCustom(random_state=42), n_jobs=4,
             param_grid={'max_depth': [5, 6, 7, 8, 9],
                         'max_features': [4, 5, 6, 7]},
             scoring='roc_auc')

In [18]:
print(f"Best param: {gs_rf_clf.best_params_}")
print(f"ROC AUC: {gs_rf_clf.score(X, y)}")

Best param: {'max_depth': 8, 'max_features': 6}
ROC AUC: 0.8495774914781747
